In [1]:
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize
import numpy as np
import matplotlib.pyplot as plt
import time



def Simulator_v3(td, tm, activity, test=False, activities=[100, 100], tss=[0, 0], final_values=False):
    '''
    This function, Simulator, calculates the number of the atoms at the measurement time, tm, for 
    the case that the person has died at time td. The unit of the time values is day.
    This function solves a set if differential equations to find the number if the atoms.
    '''
    
    decay_time = td
    time_of_measurement = tm
    
    
    lds = [0.18, 0.000085, 0.139, 0.005007]
    Activity = activity

    #--This is for calculation of # of atoms for lead over time e.g. a month
    if(test):
        N0_Rn = 0
    else:
        N0_Rn = (int(Activity) * 3.8 * 24 * 3600)/0.693
    N0_Pb = 0
    N0_Bi = (lds[1]/lds[2])*N0_Pb
    N0_Po = (lds[1]/lds[3])*N0_Pb
    
    tss = np.append(tss, td)
    def acts(t):
        for i in range(1, len(tss)):
            if(t<tss[i])and(t>=tss[i-1]):
                return activities[i-1]
        return activities[-1]
    
    
    # Funtion f(t, N) and is the function that describes the differential equation:
    # f(t, N) = dN/dt where N is an array the number of atoms for different compounds.
    def f(t, N):
        if(test):
            act = acts(t) if t<=decay_time else 0
            #N1decay = 0 if t<decay_time else lds[0]
            return [act-lds[0]*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]
        else:
            N1decay = 0 if t<decay_time else lds[0]
            return [-N1decay*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]

    
    ts = 100
    n = 500
    # C1 and C2 are the number of atoms that we measure in reality.
    time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
    
    # solve_ivp solves the set of differential equations.
    #output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12)
    output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12, t_eval=time)
    
    '''
    if final_values:
        C1 = np.interp(tm, output.t, output.y[1])
        C2 = np.interp(tm, output.t, output.y[2])
        C3 = np.interp(tm, output.t, output.y[3])
        R1 = C1/C3
        R2 = C1/C2
        return tm, R1, R2
    else:
        #ts = 2000
        #n = 500
        # C1 and C2 are the number of atoms that we measure in reality.
        #time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
        #time = np.linspace(ts, tm, n)
        C1 = np.interp(time, output.t, output.y[1])
        C2 = np.interp(time, output.t, output.y[2])
        C3 = np.interp(time, output.t, output.y[3])
        #C1 = output.y[1]
        #C2 = output.y[2]
        #C3 = output.y[3]
        R1 = C1/C3
        R2 = C1/C2
        return time, R1, R2
        #return output.t, R1, R2
    '''
    C1 = output.y[1]
    C2 = output.y[2]
    C3 = output.y[3]
    return output.t, C1/C3, C1/C2


def inverseProblemSolver3(R1t, R2t):   
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    def objective(inp, depth=0):
        td, ms, jump, tc = inp
        if(ms<=0)or(td<=1)or(tc<0)or(tc>tcs[depth])or(jump<=0.2)or(jump>1.8):
            return np.inf
        Rd_act = 100
        _, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        #_, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        #print(inp)
        return (((R1t-r1[-1])/R1t)**2 + 20*((R2t-r2[-1])/R2t)**2)**0.5
    
    for depth in range(len(tcs)):
        print(depth)
        #solution = minimize(objective, x0=[15000, 10, 0.8, 0.99*tcs[depth]], method='Nelder-Mead', tol=1e-9, args=depth)
        solution = minimize(objective, x0=[15000, 10, 0.4, 0.99*tcs[depth]], method='Nelder-Mead', args=depth, 
                           options= {'fatol': 1e-9, 'xatol': 1e-4})

        if(solution.success):
            break
    print('depth: {}'.format(depth))
    return solution

In [4]:
############ Generating data and solve with the new method ################

import numpy as np
import matplotlib.pyplot as plt


ens = 5

results = np.zeros(ens, dtype='bool')
errs = np.zeros(ens)
tms = np.zeros(ens)
total_time = 0
data = list()
for i in range(ens):
    start = time.time()
    
    td = (30 + np.random.rand()*10)*365 
    tm = np.random.rand()*21
    
    Njumps = 5 + int(np.random.rand()*15)
    Rd = 100 + 2*(np.random.rand(Njumps)-0.5)*70
    Tjumps = np.sort(np.random.rand(Njumps))*td
    Tjumps[0] = 0
        
    t, R1t, R2t = Simulator_v3(td, td+tm, 100, True, Rd, Tjumps)
    sol = inverseProblemSolver3(R1t[-1], R2t[-1])
    errs[i] = np.abs(tm-sol.x[1])
    results[i] = sol.success
    
    s = {'td':td, 'tm':tm, 'Rd':Rd, 'Tjumps':Tjumps, 'sol':sol}
    data.append(s)
    
    stop = time.time()
    print('{}/{}\t{:.2f}\t{}\t{:.2f} seconds'.format(i + 1, ens, errs[i], results[i], stop-start))
    total_time += stop-start
        
    remaining = total_time * (ens - (i+1))/(i+1)
    remtime = [int(remaining//3600), int((remaining%3600)//60), int(remaining%60)]
    print('Estimated remaining time: {} hours {} minutes {} seconds\n\n'.format(remtime[0], remtime[1], remtime[2]))
    

np.save('Data_report_no_error_v2_{}.npy'.format(int(time.time())), data)

%matplotlib qt
plt.plot(tms, np.abs(errs), '.')
plt.xlabel('Time difference between death and measurement [days]')
plt.ylabel('Estimation error [days]')
plt.grid()
plt.show()

plt.show()

0
depth: 0
1/5	0.08	True	55.72 seconds
Estimated remaining time: 0 hours 3 minutes 42 seconds


0
depth: 0
2/5	1.18	True	62.66 seconds
Estimated remaining time: 0 hours 2 minutes 57 seconds


0
depth: 0
3/5	0.01	True	79.15 seconds
Estimated remaining time: 0 hours 2 minutes 11 seconds


0
depth: 0
4/5	0.80	True	57.08 seconds
Estimated remaining time: 0 hours 1 minutes 3 seconds


0
depth: 0
5/5	0.08	True	95.02 seconds
Estimated remaining time: 0 hours 0 minutes 0 seconds




In [15]:
#################  Visualization of data without uncertainty plus mean and std  ###################
import numpy as np
import matplotlib.pyplot as plt

# Load data
#file = 'Data_report_no_error_v2_2023_12_14_4_53_30.npy' #Double measurement with 4 jumps up to 90% of lifetime (in the paper)
file = 'Data_no_jump_no_uncertainty_single_measurement_2024_4_4_19_26_34.npy' # In the paper *****(Off-axis data)
data = np.load(file, allow_pickle=True)

# Extract data
N = len(data)
errs = np.zeros(N)
tms = np.zeros(N)
for i in range(N):
    tms[i] = data[i]['tm']
    #errs[i] = (data[i]['tm']-data[i]['sol'].x[1])*24
    errs[i] = (data[i]['tm'] - data[i]['sol'].x[1]) * 24 * 3600  # seconds

# Filter
#inds = (tms <= 15) * (np.abs(errs) < 30)  # just for data less than 15 days are considered and for a range of error (to remove an off-axis data point)
inds = (tms <= 15) * (np.abs(errs) < 1)  # just for data less than 15 days are considered and for a range of error (to remove an off-axis data point)
mean = np.mean(np.abs(errs[inds]))
std = np.std(errs[inds])

# --- Shared plot area size setup ---
figsize = (5.3, 4.3)
ax_position = [0.15, 0.15, 0.8, 0.75]  # [left, bottom, width, height] as fraction of figure

# --- Scatter plot ---
fig1 = plt.figure(figsize=figsize)
ax1 = fig1.add_axes(ax_position)
ax1.scatter(tms[inds], errs[inds], marker='.')
ax1.set_xlabel('Time between death and measurement ($t_m$) [days]', fontsize=12)
ax1.set_ylabel('Estimation error [s]', fontsize=12)
ax1.set_title('Mean Abs[error] = {:.2e} s\nStd error = {:.2e} s'.format(mean, std), fontsize=12)
ax1.grid()
fig1.savefig("scatter_plot.pdf")

# --- Histogram plot ---
fig2 = plt.figure(figsize=figsize)
ax2 = fig2.add_axes(ax_position)
ax2.hist(errs[inds], bins=100)
ax2.set_xlabel('Estimation error [s]', fontsize=12)
ax2.set_ylabel('Error population', fontsize=12)
#ax2.grid()
fig2.savefig("histogram_plot.pdf")

plt.show()
